# Introduction

In this notebook we will be training a Random Forest to predict the scorline of EPL games using the existing pipeline pipeline_ex_RandomForest.

# Step 1 - Import and Data Prep

We import the necessary libraries and get the training, validation and testing datasets. These sets have undergone cleaning, preparation and feature engeneering in the original pipeline file, so they are ready to use

In [1]:
# Random Forest Evaluation Notebook

# Imports
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import sys, os

# Import pipeline and load data
sys.path.append(os.path.abspath('..'))  # Adjust if needed
from pipeline_ex_RandomForest import get_train_val_test_data

X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test_data()

c:\Users\nikol\Documents\ML\ML_group_project\pipeline_ex_RandomForest.py:190: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  db['odds_hw'] = db[home_win_cols].mean(axis=1)
c:\Users\nikol\Documents\ML\ML_group_project\pipeline_ex_RandomForest.py:192: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  db['odds_d']  = db[draw_cols].mean(axis=1)
c:\Users\nikol\Documents\ML\ML_group_project\pipeline_ex_RandomForest.py:194: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

--- NaN Check before scaling ---
NaNs in X_train: 0
NaNs in X_val: 0
NaNs in X_test: 0
--- NaN Check after scaling ---
NaNs in X_train_scaled: 0
NaNs in X_val_scaled: 0
NaNs in X_test_scaled: 0


In [2]:
# Check data types and shapes
print("X_train type:", type(X_train))
print("X_train shape:", X_train.shape)
print("y_train type:", type(y_train))
print("y_train shape:", y_train.shape)
print("First 5 y_train columns:", y_train.columns.tolist())
print("First 5 X_train columns:", X_train.columns.tolist())

X_train type: <class 'pandas.core.frame.DataFrame'>
X_train shape: (3233, 227)
y_train type: <class 'pandas.core.frame.DataFrame'>
y_train shape: (3233, 2)
First 5 y_train columns: ['FTHG', 'FTAG']
First 5 X_train columns: ['HTHG', 'HTAG', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'BbOU', 'BbMx_2.5', 'BbAv_2.5', 'BbMx_2.5', 'BbAv_2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'B365_2.5', 'B365_2.5', 'P_2.5', 'P_2.5', 'Max_2.5', 'Max_2.5', 'Avg_2.5', 'Avg_2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'B365C_2.5', 'B365C_2.5', 'PC_2.5', 'PC_2.5', 'MaxC_2.5', 'MaxC_2.5', 'AvgC_2.5', 'AvgC_2.5', 'AHCh', 'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'BFEH', 'BFED', 'BFEA', 'BFE_2.5', 'BFE_2.5', 'BFEAHH', 'BFEAHA', 'BFECH', 'BFECD', 'BFECA', 'BFEC_2.5', 'BFEC_2.5', 'BFECAHH', 'BFECAHA', 'odds_hw

# Step 2 - Hyperparameter Tuning

We tune the hyperparemeters using GridSearchCV to get the optimal model performance.

In [5]:
param_grid = {
    'estimator__n_estimators': [50, 100, 200],
    'estimator__max_depth': [3, 5, 7, None],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4],
    'estimator__max_features': ['sqrt', 'log2', None]
}

# Create time series cross-validation splits
tscv = TimeSeriesSplit(n_splits=5)

base_model = MultiOutputRegressor(RandomForestRegressor(random_state=42, n_jobs=-1))
grid = GridSearchCV(base_model, param_grid, cv=tscv, scoring='neg_root_mean_squared_error', verbose=2, n_jobs=-1)
grid.fit(X_train, y_train)

print("Best parameters:", grid.best_params_)
print("Best CV score (neg RMSE):", grid.best_score_)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best parameters: {'estimator__max_depth': 5, 'estimator__max_features': None, 'estimator__min_samples_leaf': 2, 'estimator__min_samples_split': 2, 'estimator__n_estimators': 200}
Best CV score (neg RMSE): -0.8333504182043974


# Step 4 - Validation Evaluation

We validate the model and see how it performs on the validation set. This should give us an idea of how it performs and if the hyperparameter optimization was successful

In [6]:
# Validation set evaluation with rounded predictions
y_val_pred = grid.predict(X_val.values)
y_val_pred_rounded = np.round(y_val_pred)  # Round predictions to nearest integer

print("Validation RMSE (raw):", mean_squared_error(y_val, y_val_pred))
print("Validation RMSE (rounded):", mean_squared_error(y_val, y_val_pred_rounded))
print("Validation MAE (raw):", mean_absolute_error(y_val, y_val_pred))
print("Validation MAE (rounded):", mean_absolute_error(y_val, y_val_pred_rounded))
print("Validation R2 (raw):", r2_score(y_val, y_val_pred))
print("Validation R2 (rounded):", r2_score(y_val, y_val_pred_rounded))

Validation RMSE (raw): 0.48362443704865987
Validation RMSE (rounded): 0.5705445544554455
Validation MAE (raw): 0.5690365887115081
Validation MAE (rounded): 0.5160891089108911
Validation R2 (raw): 0.6799545925509689
Validation R2 (rounded): 0.620746909881414


c:\Users\nikol\Documents\ML\ML_group_project\jupyter_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\nikol\Documents\ML\ML_group_project\jupyter_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


# Step 5 - Retrain and Evaluate

We can now retrain the model on the train+val sets to ensure it sees as many examples as possible while avoiding leakage by leaving the test set untouched. We will use the test set to evaluate performance

In [7]:
# Retrain on train+val, test on test set
X_trainval = pd.concat([X_train, X_val])
y_trainval = pd.concat([y_train, y_val])
final_model = MultiOutputRegressor(
    RandomForestRegressor(
        random_state=42,
        n_jobs=-1,
        **{k.replace('estimator__', ''): v for k, v in grid.best_params_.items()}
    )
)
final_model.fit(X_trainval.values, y_trainval.values)
y_test_pred = final_model.predict(X_test.values)

# Step 6 - Evaluation Results

Here we see the results of our testing

In [8]:
# Test set evaluation with rounded predictions
y_test_pred = final_model.predict(X_test.values)
y_test_pred_rounded = np.round(y_test_pred)  # Round predictions to nearest integer

print("Test RMSE (raw):", mean_squared_error(y_test, y_test_pred))
print("Test RMSE (rounded):", mean_squared_error(y_test, y_test_pred_rounded))
print("Test MAE (raw):", mean_absolute_error(y_test, y_test_pred))
print("Test MAE (rounded):", mean_absolute_error(y_test, y_test_pred_rounded))
print("Test R2 (raw):", r2_score(y_test, y_test_pred))
print("Test R2 (rounded):", r2_score(y_test, y_test_pred_rounded))

Test RMSE (raw): 0.4237828080590509
Test RMSE (rounded): 0.5160493827160494
Test MAE (raw): 0.5340622033383746
Test MAE (rounded): 0.4814814814814815
Test R2 (raw): 0.7216476057828166
Test R2 (rounded): 0.6614602888808354


In [9]:
# Show predictions vs actuals with rounded predictions
results = pd.DataFrame({
    'FTHG_true': y_test['FTHG'].values,
    'FTHG_pred': y_test_pred[:, 0],
    'FTHG_pred_rounded': y_test_pred_rounded[:, 0],
    'FTAG_true': y_test['FTAG'].values,
    'FTAG_pred': y_test_pred[:, 1],
    'FTAG_pred_rounded': y_test_pred_rounded[:, 1]
})
print(results.head())

   FTHG_true  FTHG_pred  FTHG_pred_rounded  FTAG_true  FTAG_pred  \
0          1   1.364469                1.0          0   1.045570   
1          5   4.334601                4.0          0   0.777992   
2          4   2.273197                2.0          2   1.540733   
3          0   0.157246                0.0          0   0.775358   
4          0   0.523431                1.0          2   1.567010   

   FTAG_pred_rounded  
0                1.0  
1                1.0  
2                2.0  
3                1.0  
4                2.0  


# Additional Visuals - Feature Importance

We can see what features were the most important and had the most weight in the predictions of the model

In [10]:
# Feature importance analysis
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': final_model.estimators_[0].feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print("\nTop 10 most important features:")
print(feature_importance.head(10))


Top 10 most important features:
                        feature  importance
0                          HTHG    0.714583
4                           HST    0.190747
80                      odds_hw    0.006149
82                      odds_aw    0.005384
108   h2h_avg_home_goals_last_5    0.005175
3                            AS    0.003859
106     defensive_solidity_home    0.003770
81                       odds_d    0.003757
93   avg_goals_conceded_home_L5    0.003154
2                            HS    0.003074
